In [39]:
# Importing required modules for maze creation and visualization
from pyamaze import maze, agent, COLOR, textLabel
from collections import deque

# ---- Step 1: Helper Function to Get Next Cell Based on Direction ----
def get_next_cell(current, direction):
    """
    Returns the coordinates of the neighboring cell based on the direction.
    Directions are 'E' (East), 'W' (West), 'S' (South), 'N' (North).
    """
    row, col = current
    if direction == 'E':  # Move East
        return (row, col + 1)
    elif direction == 'W':  # Move West
        return (row, col - 1)
    elif direction == 'S':  # Move South
        return (row + 1, col)
    elif direction == 'N':  # Move North
        return (row - 1, col)

# ---- Step 2: BFS Search Function to Explore the Maze ----
def BFS_search(maze_obj, start=None):
    """
    Perform Breadth-First Search (BFS) to explore the maze and find the path to the goal.
    
    Arguments:
    maze_obj -- The maze object to perform BFS on
    start -- The starting position (optional, default is bottom-right corner)
    
    Returns:
    exploration_order -- List of cells explored during the BFS
    visited -- Dictionary mapping each visited cell to its parent
    path_to_goal -- Dictionary with the reconstructed path to the goal
    """
    # Start position == Bottom-right corner if not specified
    if start is None:
        start = (maze_obj.rows, maze_obj.cols)

    # Initialize BFS frontier with the start point
    frontier = deque([start])

    # Dictionary to store the path taken to reach each cell
    visited = {}

    # List to track cells visited in the search process
    exploration_order = []

    # Set of explored cells to avoid revisiting
    explored = set([start])

    while frontier:
        # Dequeue the next cell to process
        current = frontier.popleft()

        # If the goal is reached, stop the search
        if current == maze_obj._goal:
            break

        # Check all four possible directions (East, West, South, North)
        for direction in 'ESNW':
            # If movement is possible in this direction (no wall)
            if maze_obj.maze_map[current][direction]:
                # Calculate the coordinates of the next cell in the direction
                next_cell = get_next_cell(current, direction)

                # If the cell hasn't been visited yet, process it
                if next_cell not in explored:
                    frontier.append(next_cell)  # Add to the frontier
                    explored.add(next_cell)     # Mark as visited
                    visited[next_cell] = current  # Record the parent (current cell)
                    exploration_order.append(next_cell)  # Track the exploration order

    # Reconstruct the path from the goal to the start using the visited dictionary
    path_to_goal = {}
    cell = maze_obj._goal
    while cell != (maze_obj.rows, maze_obj.cols):
        path_to_goal[visited[cell]] = cell
        cell = visited[cell]

    return exploration_order, visited, path_to_goal

# ---- Step 3: Create Maze and Perform BFS Search ----
# Create a 30 x 50 maze and load it from a CSV file
m = maze(30, 50)
m.CreateMaze(loadMaze='D:/Masters Projects/Master-In-AI/Foundation of Artificial Intelligence/My Project Work/maze--2024-11-30--21-36-21.csv')

# Perform BFS search on the maze and get the exploration order and paths
exploration_order, visited_cells, path_to_goal = BFS_search(m)

# ---- Step 4: Initialize Agents for Visualization ----
# Create agents to visualize the BFS search process
agent_bfs = agent(m, footprints=True, shape='square', color=COLOR.red)  # Visualize BFS search order
agent_trace = agent(m, footprints=True, shape='star', color=COLOR.yellow, filled=False)  # Full BFS path
agent_goal = agent(m, 1, 1, footprints=True, color=COLOR.blue, shape='square', filled=True, goal=(m.rows, m.cols))  # Goal agent

# ---- Step 5: Trace Paths for Agents ----
# Visualize the agents' movements along their respective paths with delays
# First, ensure that we trace the full BFS path and the path to goal

m.tracePath({agent_bfs: exploration_order}, delay=0.1)  # BFS search order path (red agent)
m.tracePath({agent_trace: path_to_goal}, delay=0.1)  # Full BFS path (yellow agent)

# ---- Step 6: Display Information About BFS Path ----
# Set the goal position as a label
goal_position = ("1, 1")  # Set the goal position (as a string)
textLabel(m, 'Goal Position', goal_position)

# Display the length of the BFS path (path from goal to start)
textLabel(m, 'BFS Path Length', len(path_to_goal) + 1)

# Display the total number of cells explored during BFS search
textLabel(m, 'BFS Search Length', len(exploration_order))

# ---- Step 7: Run the Maze Visualization ----
# Run the maze visualization with all agents and paths traced
m.run()
